In [1]:
import numpy as np
import scipy as sp
import math
from sklearn import datasets, base
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import pandas as pd
from sklearn.cross_validation import cross_val_score
from copy import deepcopy
import itertools
classifiers = [
    KNeighborsClassifier(n_jobs = -1),
    SVC(probability = True),
#     GaussianProcessClassifier(n_jobs = -1),
    DecisionTreeClassifier(),
    RandomForestClassifier()]

# classifiers_name = ["KNN", "SVM", "GP", "DT", "RF"]
classifiers_name = ["KNN", "SVM", "DT", "RF"]
beta = "XGB"

parameters = {"KNN":{"n_neighbors":[]},
              "SVM":{"C":[],"kernel":[],"degree":[]},
              "GP":{"kernel":[],"n_restarts_optimizer":[]},
              "DT":{"splitter":[], "max_depth":[]},
              "RF":{"n_estimators":[], "max_depth":[] },
              "XGB":{"learning_rate":[], "n_estimators":[], "max_depth":[],"min_child_weight":[]}
            }
classifier_parameters = []

iris = datasets.load_iris()
X = iris.data
Y = iris.target
Df = pd.DataFrame()
Df["X"] = X.tolist()
Df["Y"] = Y.tolist()

/home/neo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def sample_hp_Ai(Ai):
    global classifier_parameters
    if(Ai is "KNN"):
        parameters["KNN"]["n_neighbors"] = math.ceil(np.random.uniform(100))
        classifier_parameters.append(parameters["KNN"].copy())
        return(parameters["KNN"])
    elif(Ai is "SVM"):
        parameters["SVM"]["C"] = float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0))))
        parameters["SVM"]["kernel"] = str(np.random.choice(["linear", "rbf"]))
        parameters["SVM"]["degree"] = np.random.randint(1,4)
        classifier_parameters.append(parameters["SVM"].copy())
        return(parameters["SVM"])
    elif(Ai is "GP"):
        parameters["GP"]["kernel"] = str(np.random.choice(["linear", "rbf"]))
        parameters["GP"]["n_restarts_optimizer"] = np.random.randint(1,10)
        classifier_parameters.append(parameters["GP"].copy())
        return(parameters["GP"])
    elif(Ai is "DT"):
        parameters["DT"]["splitter"] = np.random.choice(["best", "random"])
        parameters["DT"]["max_depth"] = np.random.randint(1,100)
        classifier_parameters.append(parameters["DT"].copy())
        return(parameters["DT"])
    elif(Ai is "RF"):
        parameters["RF"]["n_estimators"] = np.random.randint(1,20)
        parameters["RF"]["max_depth"] = np.random.randint(1,100)
        classifier_parameters.append(parameters["RF"].copy())
        return(parameters["RF"])
    elif(Ai is "XGB"):
        parameters["XGB"]["learning_rate"].append(float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0)))))
        parameters["XGB"]["n_estimators"].append(np.random.randint(800,2000))
        parameters["XGB"]["max_depth"].append(np.random.randint(5,200))
        parameters["XGB"]["min_child_weight"].append(np.random.randint(1,10))
        classifier_parameters.append(parameters["XGB"].copy())
        return(parameters["XGB"])

In [3]:
def get_params(A, B, P, N):
    mu = np.random.dirichlet(P)
    N = [int(mu_i*N) for mu_i in mu]
    [sample_hp_Ai(classifiers_name[i]) for i in range(len(classifiers_name)) for j in range(N[i])]
    Phi = sample_hp_Ai("XGB")
    Psi = [parameters[key] for key in parameters if key is not "XGB"]
    return Phi, N, Psi

In [4]:
Phi, N, Psi = get_params(classifiers_name, "RF", P=(np.arange(1,len(classifiers_name)+1)*100)[::-1], N=10)

In [5]:
class Blend:
    def __init__(self, A = ["KNN", "SVM", "GP", "DT", "RF"], B = "XGB",D = [], L = 2, Phi = [], Psi = [], N=100 ):
        self.A = A
        self.B = B
        self.D = D
        self.L = L
        self.Phi = Phi
        self.Psi = Psi
        self.N = N
                    
    def fit(self, A, B, D, L, Phi, N, Psi):
#         p = np.random.normal(loc=0.7, scale=0.1)
        p=0.7
        D_fw = pd.DataFrame()
        M = []
        F = []
        G = []
        for l in range(1,L+1):
            D_dash = D.sample(frac=p)
            orig_index = sorted(Df.index.tolist())
            train_index = sorted(D_dash.index.tolist())
            test_index = sorted(set(orig_index) - set(train_index))
            D_dash_bar = Df.ix[test_index]
            classifier_index = 0
            for i in range(0,len(classifiers_name)):
                temp = []
                if(N[i]):
                    for j in range(0,N[i]):
                        temp_clf = None
                        temp_clf = classifiers[i].set_params(**classifier_parameters[classifier_index])
                        temp_clf = temp_clf.fit(X = D_dash["X"].tolist(), y = D_dash["Y"].tolist())
                        temp.append(deepcopy(temp_clf))
                        classifier_index += 1
                M.append(temp)
            for row in D_dash_bar["X"]:
                F_temp = []
                G_temp = []
                for models in M:
                    for model in models:
                        M_bc = model.predict_proba([row])
                        F_temp += [xa*ele for xa in row for ele in M_bc[0]]
                        G_temp += [ele for ele in M_bc[0]]
                F.append(F_temp)
                G.append(G_temp)
            d_assign = [list(itertools.chain.from_iterable(ele)) for ele in \
                        list(zip(D_dash_bar["X"].tolist().copy(),F.copy(),G.copy()))]
            D_fw["X"] = d_assign
            D_fw["Y"] = D_dash_bar["Y"].tolist().copy()
            D_fw = D_fw.reset_index()
            
            Beta = xgb.XGBClassifier()
            Beta.set_params(**classifier_parameters[-1])
#             Beta = Beta.fit(X=D_fw["X"], y=D_fw["Y"])
        return(M,D_dash,F,G, D_fw, Beta)

In [6]:
def blending_ensemble(A, B, D, P, N, k, R):
    L = 2
    M = []
    for r in range(1,R+1):
        Phi_r, N_r, Psi_r = get_params(A, B, P, N=10)
        eps_r = cross_val_score(Blend(), A, B, D, L, Phi, Psi, N, cv=k)
        print(eps_r)

In [7]:
b =Blend()

In [8]:
M_temp, D_dash, F, G, D_fw, Beta = b.fit(A=classifiers_name, B="XGB", D=Df, L=1, Phi=Phi, N=N, Psi=Psi)

/home/neo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [11]:
Beta = xgb.XGBClassifier()
Beta.set_params(**classifier_parameters[-1])

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=[0.02], max_delta_step=0, max_depth=[149],
       min_child_weight=[8], missing=None, n_estimators=[967], nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [14]:
XX = D_fw["X"]
YY = D_fw["Y"]

In [18]:
Beta.fit(XX,YY)

IndexError: tuple index out of range

In [22]:
SS = SVC(**classifier_parameters[3])

In [23]:
SS.fit(XX,YY)

ValueError: setting an array element with a sequence.